# ИУ5-21М Саватюгина Юлия РК2 

In [1]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score 
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
#import seaborn as sns
from collections import Counter
from sklearn.datasets import fetch_20newsgroups
import matplotlib.pyplot as plt

In [2]:
categories = ['talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc',
 'misc.forsale'
             ]
newsgroups = fetch_20newsgroups(subset='train', categories=categories)
data = newsgroups['data']

In [3]:
data

["From: eshneken@ux4.cso.uiuc.edu (Edward A Shnekendorf)\nSubject: Re: was:Go Hezbollah!!\nOrganization: University of Illinois at Urbana\nLines: 17\n\nmafifi@eis.calstate.edu (Marc A Afifi) writes:\n\n>Let's not forget that the soldiers were killed not murdered. The\n>distinction is not trivial. Murder happens to innocent people, not people\n>whose line of work is to kill or be killed. It just so happened that these\n>soldiers, in the line of duty, were killed by the opposition. \n\nThat still doesn't mean we should cheer their deaths.  Policemen are also in \nthe line of fire and their job includes the possibility of getting killed.  \nShould we be happy when they die?  As I said before, the question is not\nwhether or not you agree with the policies of Israel.  You may wish for the\nIsraelis to cease occupation, but don't rejoice in death.\n\n>-marc\n\nEd.\n\n",
 'From: jake@bony1.bony.com (Jake Livni)\nSubject: Re: was:Go Hezbollah!\nOrganization: The Department of Redundancy Depar

In [4]:
def accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray) -> Dict[int, float]:
    """
    Вычисление метрики accuracy для каждого класса
    y_true - истинные значения классов
    y_pred - предсказанные значения классов
    Возвращает словарь: ключ - метка класса, 
    значение - Accuracy для данного класса
    """
    # Для удобства фильтрации сформируем Pandas DataFrame 
    d = {'t': y_true, 'p': y_pred}
    df = pd.DataFrame(data=d)
    # Метки классов
    classes = np.unique(y_true)
    # Результирующий словарь
    res = dict()
    # Перебор меток классов
    for c in classes:
        # отфильтруем данные, которые соответствуют 
        # текущей метке класса в истинных значениях
        temp_data_flt = df[df['t']==c]
        # расчет accuracy для заданной метки класса
        temp_acc = accuracy_score(
            temp_data_flt['t'].values, 
            temp_data_flt['p'].values)
        # сохранение результата в словарь
        res[c] = temp_acc
    return res

def print_accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray):
    """
    Вывод метрики accuracy для каждого класса
    """
    accs = accuracy_score_for_classes(y_true, y_pred)
    if len(accs)>0:
        print('Метка \t Accuracy')
    for i in accs:
        print('{} \t {}'.format(i, accs[i]))

In [5]:
vocabVect = CountVectorizer()
vocabVect.fit(data)
corpusVocab = vocabVect.vocabulary_
print('Количество сформированных признаков - {}'.format(len(corpusVocab)))

Количество сформированных признаков - 41856


In [6]:
for i in list(corpusVocab)[1:10]:
    print('{}={}'.format(i, corpusVocab[i]))

eshneken=15971
ux4=39487
cso=12647
uiuc=38720
edu=15176
edward=15187
shnekendorf=34517
subject=36293
re=31648


In [7]:
test_features = vocabVect.transform(data)
test_features

<2537x41856 sparse matrix of type '<class 'numpy.int64'>'
	with 457637 stored elements in Compressed Sparse Row format>

In [8]:
# Размер нулевой строки
len(test_features.todense()[0].getA1())

41856

In [9]:
vocabVect.get_feature_names()[100:120]

['021154',
 '02139',
 '02180',
 '02215',
 '022218',
 '022222',
 '02238',
 '022903',
 '022926',
 '023039',
 '0235',
 '023937',
 '024646',
 '024850',
 '024949',
 '025031',
 '025258',
 '025426',
 '025636',
 '025818u28037']

In [10]:
def VectorizeAndClassify(vectorizers_list, classifiers_list):
    for v in vectorizers_list:
        for c in classifiers_list:
            pipeline1 = Pipeline([("vectorizer", v), ("classifier", c)])
            score = cross_val_score(pipeline1, newsgroups['data'], newsgroups['target'], scoring='accuracy', cv=3).mean()
            print('Векторизация - {}'.format(v))
            print('Модель для классификации - {}'.format(c))
            print('Accuracy = {}'.format(score))
            print('===========================')

In [11]:
vectorizers_list = [CountVectorizer(vocabulary = corpusVocab), TfidfVectorizer(vocabulary = corpusVocab)]
classifiers_list = [LogisticRegression(), MultinomialNB()]
VectorizeAndClassify(vectorizers_list, classifiers_list)

/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mod

Векторизация - CountVectorizer(vocabulary={'00': 0, '000': 1, '0000': 2, '000007': 3,
                            '0000vec': 4, '0001': 5, '000152': 6, '000246': 7,
                            '0004': 8, '000413': 9, '0005111312': 10,
                            '0005111312na3em': 11, '000k': 12, '000s': 13,
                            '000th': 14, '001': 15, '0010': 16, '001116': 17,
                            '001319': 18, '001338': 19, '001428': 20, '002': 21,
                            '002118': 22, '002302': 23, '002339': 24,
                            '002811': 25, '003': 26, '003522': 27, '003848': 28,
                            '004': 29, ...})
Модель для классификации - LogisticRegression()
Accuracy = 0.9377140832132649
Векторизация - CountVectorizer(vocabulary={'00': 0, '000': 1, '0000': 2, '000007': 3,
                            '0000vec': 4, '0001': 5, '000152': 6, '000246': 7,
                            '0004': 8, '000413': 9, '0005111312': 10,
                      

# Лучшая точность у CountVectorizer с MultinomialNaiveBayes